# Inversion and optimisation: Assessment 2 general feedback

Overall, students who completed this assignment did well. Some impressive results were achieved in both parts 2 and 3. Below is a non-exhaustive list of notes regarding good practices for each section, in addition to the mention of common mistakes.

It should be emphasized that clarity and quality of code (well structured and commented where appropriate) along with a clear presentation of results were important in all sections.

### Part 1

This section of the assignment was a straightforward and hence only limited discussion is required. A few observations and common (albeit minor) errors are noted below.

**(i)**: Data could be pre-read (i.e. data from all files read into memory prior to the FWI iteration loop) or loaded on the fly. Owing to the small size of the data, either approach was perfectly acceptable here. Note however that in large scale FWI frameworks shot data will generally be read on-the-fly since each shot will require a non-negligible amount of memory. It's worth noting that many students stored `d_obs` as a `numpy` array. This approach was fine, but for tidiness the sparse function `d_obs`, that had already been defined, should have been deleted in this case. 

(Also, storing the data in the receiver object (via `d_obs.data[:] = ...`) would have allowed for interpolating onto different time axes later in part 2(ii).

**(ii)**: Nothing much to say here. Only to note that since this notebook was not suitable for running in parallel the `if residual.grid.distributor.is_parallel` clause need not be included.

### Part 2

**(i)**: In this part it was required to replace the existing `alpha = .5 / mmax(update)` with something of the form `alpha = my_line_search(...)`, where `my_line_search(...)` was a suitable function using methods from lecture 4 to choose a more optimal value of `alpha`. It was then required to discuss and explain the algorithm being used. A few notes (and common mistakes) regarding this section:
- The `simple_line_search` function provided in lecture 4 could be used as the basis for this implementation. However, as discussed below, care needed to be taken with various aspects of the implementation.
- The implementation required the residual to be computed for different models. This could be achieved by taking a `deepcopy` of the existing. However, a slightly more efficient method, would be to store the appropriate `model.vp` in a numpy array and then updating/resetting `vp` as appropriate. This avoids needing to create a new model object. Some common mistake were
    - Continually updating the `model.vp` within each line search and not resetting `vp` after each iteration i.e. computing $\phi(\mathbf{m}+(\alpha^{(1)}+...+\alpha^{(n)})\nabla_{\mathbf{m}}\phi)$ instead of $\phi(\mathbf{m}+\alpha^{(n)}\nabla_{\mathbf{m}}\phi)$.
    - Not applying Armijo's rule correctly - note that $\lambda$ in the lecture 4 notes was `alpha` in the code you were provided with.
- Also note that you were provided with a function to compute the gradient and residual simultaneously. Optimizing `alpha` however only needed a recalculation of the residual. A nice option here was to modify the function you were given to allow a flag such as `residual_only` to be passed and then adding appropriate statements to the existing function to minimize computational cost.
- In explaining the algorithm it should have been stated clearly how the algorithm applies to the functions you're using e.g. the $f$ from lecture 4 was now $\phi$, $\mathbf{p}$ was now $\nabla_{\mathbf{m}}\phi$ and so forth. Almost no student did this. Along with this, a concise explanation regarding how the algorithm worked, tailored to this particular case, should have been given.

The affect of implementing a line search algorithm was as follows. In the absence of a method for setting `alpha` in a more optimal fashion, after a certain number of FWI iterations the residual would start to oscillate wildly. Selecting `alpha` via a line search results in the gradient descent method being far more stable when the number of FWI iterations is large (that is, the gradient descent algorithm will not overshoot the minimum in such an extreme fashion).

**(ii)**: Synthetic shot data was being provided in this section, meaning the number of shots, number of receivers and source and receiver locations were fixed. Since the exact method by which the synthetic data was generated was an unknown, some things worth trying were the following:
- Using a larger grid size (but ensuring physical source and receiver locations didn't change). Note that in theory grid spacing could also have been modified. However, this would then require a smaller time step in order to satisfy the CFL condition and hence interpolation of the underlying shot data onto a new time axis (which was not expected in the limited time available to complete this exercise).
- Using solvers with a higher space order.
- More boundary layers.
- More FWI iterations.
- Choosing a different starting model, e.g. using your guess of the underlying model as a starting point.

Other well justified modifications not listed above were of course acceptable.

Additionally, an even better method of computing `alpha` (e.g. using `scipy.optimize.minimize` or a self implemented conjugate gradient scheme) could have been used here. But such additions were better kept for part 3.

For each modification, a concise explanation of why you expect the change to improve the result should be given. For example:
**Inceasing the number of boundary layers:** Owing to the small size of the computational domain and original boundary layer width, reflected wave-amplitudes remained significant when compared to the amplitude of the original wave-front resulting in pollution of the recorded wave-field at the receivers. Increasing the number of boundary layers to 20 resulted in reflected amplitudes dropping by over a half resulting in cleaner data at the receivers and hence potentially better inversion results.

Note that owing to how the synthetics were generated, the two modifications that most readily led to better inversion results were
1) Increasing the number of FWI iterations.
2) More optimal gradient descent updates (through a successful implementation of the line search algorithm).

An image of the actual underlying model is below:
![Part 1/2 underlying model](model.png)

### Part 3

This section required you to modifications similar to those made in part 2. While many of the suitable modifications that could be made here are discussed in the part 2 feedback above, and important aspect of this part is that, since you were generating the synthetics you were afforded a much greater degree of freedom.

One interesting avenue I was hoping people would explore was to allow for a greater decoupling between the synthetic data and the modelled data. That is, for example, generating the true model data with a larger number of boundary layers and a higher space order than the guess model (i.e. trying to mimic the higher fidelity of real data somehow). Then, you could explore the affect of increasing parameters such as space order in a *slightly* less artificial manner.

Some modifications that (could) lead to a better inversion results are:
- More FWI iterations
- More shots
- Larger grid size
- Finer grid spacing
- More receivers (but only if the grid spacing is decreased and/or their locations modified)
- More boundary layers
- Implement an algorithm to determine a more optimal `alpha` (line search from part 2 or other more advanced algorithm(s)).

Some modifications that would have a lesser affect
- Playing with source and receiver locations
- Increasing space order of the model. Note that in a real situation this would be a significant factor. Here, owing to the manner in which the artificial data is generated and the simplicity of the underlying model it wont play such a significant role. If this is decoupled between the synthetic and modelled data however (as mentioned discuss above), it will make a greater difference.

A common mistake in this section was not providing a physical and mathematical reasoning behind modifications being made.

Another common mistake was judging the quality of the inversion results vs the absolute value of the residual. Results should be judged against a suitably normalized residual since the absolute value of the residual is dependent on the number of grid points in the model: more grid points = higher residual and hence a 'larger' residual can equate to a better result if comparing to that on a smaller grid of course.